In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import rcca_modified as rcca
#reload(rcca)
from unils_cca import set_diag_to_zero
from metrics import ClassificationResult
from unils_cca import binarize, set_diag_to_zero, BuildMatrixA, printMatrix, print_q_val_hist, zscore, plot_zscore, BuildFeatureVector,\
train_test_2D, build_distance_for_node,  get_features, build_distance_matrix, remove_ambiguous, \
get_pairs_distance_matched,  reshape_2d_to_1d, demean
import copy
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score


In [2]:
InteractionsFileCaptureC='/srv/scratch/mtaranov/CaptureC_ChicagoCalls/CaptureC_SC_bait-bait.bed.gz'
#InteractionsFileCaptureC='/mnt/lab_data/kundaje/mtaranov/ChicagoCalls/D0_D2D8_merge_BaitToBait_intra.bed.gz'
PromoterFile='/srv/scratch/mtaranov/HindIII/PromoterCapture_Digest_Human_HindIII_baits_ID.bed'
DATADIR='/users/mtaranov/CCA_model/data/'
DATADIR_CaptureC = DATADIR+'CaptureC/'
DATADIR_kernels = DATADIR+'CaptureC_kernels/'
DATADIR_nodes = DATADIR+'nodes/'
DATADIR_features = DATADIR+'features/'
DATADIR_preds = DATADIR+'preds_2d/'
DATADIR_weights = DATADIR+'weights/'

## Train model, save weights

In [3]:
all_chrs = ['chr'+str(i+1) for i in range(22)]+['chrX']

In [4]:
best_reg=0.001
best_numCC=50
cca = rcca.CCA(reg=best_reg, numCC=best_numCC)

for chr in all_chrs:
    train_kernel_CaptureC = np.load(DATADIR_kernels+'kernel_train_CaptureC_'+chr+'.npy')
    train_features = np.load(DATADIR_features+'train_features_'+chr+'.npy')
    cca_train = cca.train([demean(train_kernel_CaptureC), demean(train_features)])
    np.save(DATADIR_weights+'ws_train_'+chr, cca_train.__dict__['ws'][1])

Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 comp

## Save predictions, 2d

In [5]:
for chr in all_chrs:
    test_features = np.load(DATADIR_features+'test_features_'+chr+'.npy')
    ws = np.load(DATADIR_weights+'ws_train_'+chr+'.npy')
    preds, projs = cca.predict_from_features(test_features, ws)
    np.save(DATADIR_preds+'pred_test_2d_'+chr, set_diag_to_zero(preds))

## Concatenate test data from all chrs

In [6]:
labels_test_all = np.empty((0,))
preds_test_all = np.empty((0,))
pair_dist_test_all = np.empty((0,))

for chr in all_chrs:
    labels_test_2d = np.load(DATADIR_CaptureC+'test_CaptureC_'+chr+'.npy')
    preds_test_2d = np.load(DATADIR_preds+'pred_test_2d_'+chr+'.npy')
    test_nodes = np.load(DATADIR_nodes+'test_nodes_'+chr+'.npy')
    pair_dist_2d = build_distance_matrix(PromoterFile, chr)
    pair_dist_test_2d = pair_dist_2d[test_nodes][:,test_nodes]
    
    labels_test_1d = reshape_2d_to_1d(labels_test_2d)
    preds_test_1d = reshape_2d_to_1d(preds_test_2d)
    pair_dist_test_1d = reshape_2d_to_1d(pair_dist_test_2d)
    
    labels_test_all = np.concatenate((labels_test_all, labels_test_1d), axis=0)
    preds_test_all = np.concatenate((preds_test_all, preds_test_1d), axis=0)
    pair_dist_test_all = np.concatenate((pair_dist_test_all, pair_dist_test_1d), axis=0)
    
    labels_noAmbig, preds_noAmbig, pair_dist_noAmbig = remove_ambiguous(labels_test_all, preds_test_all, pair_dist_test_all, thres=10)

## Get distance matched labels and preds from test data 

In [7]:
labels_dist_matched, preds_dist_matched = get_pairs_distance_matched(labels_test_all, preds_test_all, pair_dist_test_all, 10000, 2000000, 10000, 1, 10)

distance window:  [ 10000 , 20000 ]
number of pos at distance= 16
number of neg at distance= 104
labels at dist:  (32,)
preds at dist:  (32,)
distance window:  [ 20000 , 30000 ]
number of pos at distance= 10
number of neg at distance= 18
labels at dist:  (20,)
preds at dist:  (20,)
distance window:  [ 30000 , 40000 ]
number of pos at distance= 2
number of neg at distance= 4
labels at dist:  (4,)
preds at dist:  (4,)
distance window:  [ 40000 , 50000 ]
more pos than neg
number of pos at distance= 10
number of neg at distance= 6
labels at dist:  (12,)
preds at dist:  (12,)
distance window:  [ 50000 , 60000 ]
more pos than neg
number of pos at distance= 10
number of neg at distance= 2
labels at dist:  (4,)
preds at dist:  (4,)
distance window:  [ 60000 , 70000 ]
number of pos at distance= 4
number of neg at distance= 4
labels at dist:  (8,)
preds at dist:  (8,)
distance window:  [ 70000 , 80000 ]
number of pos at distance= 4
number of neg at distance= 6
labels at dist:  (8,)
preds at dist

In [8]:
print labels_dist_matched.shape, preds_dist_matched.shape

(452, 1) (452, 1)


# Model Evaluation

## Performance on test data for each chr / weights from individual chrs

In [10]:
ROC=[]
PRC=[]
for chr in all_chrs:
    labels = np.load(DATADIR_CaptureC+'test_CaptureC_'+chr+'.npy')
    preds = np.load(DATADIR_preds+'pred_test_2d_'+chr+'.npy')
    y=binarize(labels).astype(int).reshape(labels.shape[0]*labels.shape[0],1)
    y_pred=preds.reshape(preds.shape[0]*preds.shape[0],1)
    print chr
    print(ClassificationResult(y, y_pred))
    ROC.append(roc_auc_score(y, y_pred))
    precision, recall = precision_recall_curve(y, y_pred)[:2]
    PRC.append(auc(recall, precision))
print "Average across all chrs: ROC=", sum(ROC)/len(ROC), " PRC=", sum(PRC)/len(PRC)

chr1
Balanced Accuracy: 55.03%	auROC: 0.646	 auPRC: 0.041
Recall at 5% | 10% | 25% FDR: 0.0% | 1.0% | 3.0%	Num Positives: 3772	 Num Negatives: 461352

chr2
Balanced Accuracy: 53.92%	auROC: 0.634	 auPRC: 0.051
Recall at 5% | 10% | 25% FDR: 2.5% | 2.8% | 3.5%	Num Positives: 1862	 Num Negatives: 230462

chr3
Balanced Accuracy: 53.92%	auROC: 0.637	 auPRC: 0.035
Recall at 5% | 10% | 25% FDR: 0.7% | 0.7% | 1.6%	Num Positives: 1824	 Num Negatives: 158176

chr4
Balanced Accuracy: 53.20%	auROC: 0.622	 auPRC: 0.066
Recall at 5% | 10% | 25% FDR: 2.7% | 2.9% | 4.9%	Num Positives: 822	 Num Negatives: 80403

chr5
Balanced Accuracy: 54.96%	auROC: 0.620	 auPRC: 0.038
Recall at 5% | 10% | 25% FDR: 0.0% | 0.0% | 2.2%	Num Positives: 1380	 Num Negatives: 111516

chr6
Balanced Accuracy: 59.69%	auROC: 0.684	 auPRC: 0.059
Recall at 5% | 10% | 25% FDR: 1.0% | 2.4% | 3.0%	Num Positives: 1962	 Num Negatives: 133462

chr7
Balanced Accuracy: 54.57%	auROC: 0.645	 auPRC: 0.074
Recall at 5% | 10% | 25% FDR: 3.9% | 4

### on distance-matched test / all chrs [10kb,2Mb]

In [11]:
print(ClassificationResult(labels_dist_matched, preds_dist_matched))

Balanced Accuracy: 53.98%	auROC: 0.527	 auPRC: 0.504
Recall at 5% | 10% | 25% FDR: 0.0% | 0.0% | 0.0%	Num Positives: 226	 Num Negatives: 226



### on non-ambig test (thres > 10) / all chrs [all]

In [12]:
print(ClassificationResult(np.expand_dims(binarize(labels_noAmbig).astype(int), axis=1), np.expand_dims(preds_noAmbig, axis=1)))

Balanced Accuracy: 57.37%	auROC: 0.676	 auPRC: 0.003
Recall at 5% | 10% | 25% FDR: 0.0% | 0.0% | 0.0%	Num Positives: 262	 Num Negatives: 2269841



### on all test (thes >0) / all chrs [all]

In [13]:
print(ClassificationResult(np.expand_dims(binarize(labels_test_all).astype(int), axis=1), np.expand_dims(preds_test_all, axis=1)))

Balanced Accuracy: 55.06%	auROC: 0.642	 auPRC: 0.052
Recall at 5% | 10% | 25% FDR: 0.0% | 1.8% | 2.8%	Num Positives: 31528	 Num Negatives: 2269841



### [10kb, 0.5Mb] with subsampled negative 

In [14]:
labels_dist_matched_lt_halfMb, preds_dist_matched_lt_halfMb = get_pairs_distance_matched(labels_test_all, preds_test_all, pair_dist_test_all, 10000, 510000, 500000, 1, 10)
print(ClassificationResult(labels_dist_matched_lt_halfMb, preds_dist_matched_lt_halfMb))

distance window:  [ 10000 , 510000 ]
number of pos at distance= 230
number of neg at distance= 2974
labels at dist:  (460,)
preds at dist:  (460,)
Balanced Accuracy: 51.96%	auROC: 0.539	 auPRC: 0.514
Recall at 5% | 10% | 25% FDR: 0.0% | 0.0% | 0.0%	Num Positives: 230	 Num Negatives: 230



### [10kb, 2Mb] with subsampled negative

In [15]:
labels_dist_matched_gt_halfMb, preds_dist_matched_gt_halfMb = get_pairs_distance_matched(labels_test_all, preds_test_all, pair_dist_test_all, 10000, 2010000, 2000000, 1, 10)
print(ClassificationResult(labels_dist_matched_gt_halfMb, preds_dist_matched_gt_halfMb))

distance window:  [ 10000 , 2010000 ]
number of pos at distance= 260
number of neg at distance= 58272
labels at dist:  (520,)
preds at dist:  (520,)
Balanced Accuracy: 52.69%	auROC: 0.550	 auPRC: 0.553
Recall at 5% | 10% | 25% FDR: 2.3% | 2.3% | 2.3%	Num Positives: 260	 Num Negatives: 260



### [10kb, 500Mb] with subsampled negative

In [16]:
labels_dist_matched, preds_dist_matched = get_pairs_distance_matched(labels_test_all, preds_test_all, pair_dist_test_all, 10000, 500010000, 500000000, 1, 10)
print(ClassificationResult(labels_dist_matched, preds_dist_matched))

distance window:  [ 10000 , 500010000 ]
number of pos at distance= 262
number of neg at distance= 2263054
labels at dist:  (524,)
preds at dist:  (524,)
Balanced Accuracy: 60.11%	auROC: 0.711	 auPRC: 0.719
Recall at 5% | 10% | 25% FDR: 5.3% | 13.7% | 41.2%	Num Positives: 262	 Num Negatives: 262



In [19]:
ROC=[]
PRC=[]
for chr in all_chrs:
    labels = np.load(DATADIR_kernels+'kernel_test_CaptureC_'+chr+'.npy')
    preds = np.load(DATADIR_preds+'pred_test_2d_'+chr+'.npy')
    y=binarize(labels).astype(int).reshape(labels.shape[0]*labels.shape[0],1)
    y_pred=preds.reshape(preds.shape[0]*preds.shape[0],1)
    print chr
    print(ClassificationResult(y, y_pred))
    ROC.append(roc_auc_score(y, y_pred))
    precision, recall = precision_recall_curve(y, y_pred)[:2]
    PRC.append(auc(recall, precision))
print "Average across all chrs: ROC=", sum(ROC)/len(ROC), " PRC=", sum(PRC)/len(PRC)

chr1
Balanced Accuracy: 49.09%	auROC: 0.488	 auPRC: 0.987
Recall at 5% | 10% | 25% FDR: 100.0% | 100.0% | 100.0%	Num Positives: 459700	 Num Negatives: 5424

chr2
Balanced Accuracy: 56.13%	auROC: 0.702	 auPRC: 0.694
Recall at 5% | 10% | 25% FDR: 4.5% | 7.1% | 18.5%	Num Positives: 118962	 Num Negatives: 113362

chr3
Balanced Accuracy: 52.68%	auROC: 0.594	 auPRC: 0.548
Recall at 5% | 10% | 25% FDR: 0.6% | 0.7% | 2.8%	Num Positives: 74174	 Num Negatives: 85826

chr4
Balanced Accuracy: 54.60%	auROC: 0.657	 auPRC: 0.465
Recall at 5% | 10% | 25% FDR: 1.0% | 2.2% | 5.8%	Num Positives: 25745	 Num Negatives: 55480

chr5
Balanced Accuracy: 55.43%	auROC: 0.683	 auPRC: 0.649
Recall at 5% | 10% | 25% FDR: 1.0% | 2.5% | 12.9%	Num Positives: 55072	 Num Negatives: 57824

chr6
Balanced Accuracy: 56.40%	auROC: 0.722	 auPRC: 0.684
Recall at 5% | 10% | 25% FDR: 0.5% | 3.3% | 18.8%	Num Positives: 65592	 Num Negatives: 69832

chr7
Balanced Accuracy: 46.95%	auROC: 0.510	 auPRC: 0.986
Recall at 5% | 10% | 25% 

/users/mtaranov/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/users/mtaranov/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.